# Solve 20051

Sort out the negative interest income problem in 20051

In [2]:
import pandas as pd
from os.path import join
from itertools import takewhile, repeat
from functools import partial
from utils.iter import pop
from utils.excel import fileToLines
from toolz.functoolz import compose

In [8]:
baseDir = r'C:\Users\steven.zhang\Desktop\20051'

In [9]:
"""
    [String] file => ([List] headers, [Iterator] subsequent lines)
"""
getLinesFromFile = compose(
    lambda lines: (pop(lines), lines)
  , partial(takewhile, lambda line: len(line) > 0 and line[0] != '')
  , fileToLines
)


getDataFromLines = compose(
    lambda t: pd.DataFrame(t[1], columns=t[0])
  , getLinesFromFile
)


getInvestmentFile = lambda dt: join(baseDir, '20051 investment position ' + dt + '.xlsx')

getProfitLossFile = lambda dt: join(baseDir, '20051 profit loss ' + dt + '.xlsx')


"""
    [String] dt (yyyymmdd) => [DataFrame] df
"""
getInvestmentPosition = compose(
    getDataFromLines
  , getInvestmentFile
)


"""
    [String] dt (yyyymmdd) => [DataFrame] df
"""
getProfitLoss = compose(
    getDataFromLines
  , getProfitLossFile
)

In [31]:
select = lambda df: df[df['Invest'] == 'XS1376566714 HTM']


"""
    [DataFrame] df => [DateFrame] df
    
    Add 'Date' column to df
"""
addDate = lambda dt, df: \
    pd.concat([df, pd.DataFrame(repeat([dt], len(df)), columns=['Date'], index=df.index)], axis=1)


"""
    [String] dt (yyyymmdd) => [DataFrame] df
"""
getPlData = lambda dt: \
compose(
    partial(addDate, dt)
  , lambda t: select(t[0]).merge(t[1], left_on='Invest', right_on='InvestID')[['Invest_x', 'Interest', 'AccruedInterest', 'TotalPAndL']]
  , lambda dt: (getProfitLoss(dt), getInvestmentPosition(dt))
)(dt)

getPlData('20200413')

,Invest_x,Interest,AccruedInterest,TotalPAndL,Date
0,XS1376566714 HTM,367916.07,645387.3,300190.91,20200413


In [32]:
pd.concat(map(getPlData, ['20200413', '20200414', '20200626']))

,Invest_x,Interest,AccruedInterest,TotalPAndL,Date
0,XS1376566714 HTM,367916.07,645387.30,300190.91,20200413
0,XS1376566714 HTM,-273886.21,3585.02,-344110.90,20200414
0,XS1376566714 HTM,-243708.73,33762.50,-253432.69,20200626


In [34]:
getCashPlData = lambda dt: \
compose(
    partial(addDate, dt)
  , lambda df: df[df['Invest'] == 'USD'][['Invest', 'TotalPAndL']]
  , getProfitLoss
)(dt)

x = getCashPlData('20200413')

In [35]:
pd.concat(map(getCashPlData, ['20200413', '20200414', '20200626']))

,Invest,TotalPAndL,Date
29,USD,-117717.17,20200413
30,USD,-115849.02,20200414
55,USD,-145529.97,20200626
